In [16]:
import numpy
import time

import pycuda
import pycuda.autoinit
import pycuda.compiler
import pycuda.gpuarray

from pycuda.driver import In, Out, InOut

In [77]:
module = pycuda.compiler.SourceModule("""
__global__ void reduce_a(int lenparents, int d, int* parents, float* mutablescan) {
    unsigned int i = threadIdx.x + blockIdx.x*blockDim.x;
    if (i < lenparents  &&  i >= d  &&  parents[i] == parents[i - d]) {
        mutablescan[i] = mutablescan[i] + mutablescan[i - d];
    }
    __syncthreads();
}

__global__ void reduce_b(int lenstarts, int* starts, int* stops, float* scan, float* output) {
    unsigned int i = threadIdx.x + blockIdx.x*blockDim.x;
    if (i < lenstarts) {
        if (starts[i] == stops[i]) {
            output[i] = 0.0;
        }
        else {
            output[i] = scan[stops[i] - 1];
        }
    }
}
""")

reduce_a = module.get_function("reduce_a")
reduce_b = module.get_function("reduce_b")

In [78]:
tasksize = 100000000
averagesize = 5

# counts to offsets to starts/stops
counts = numpy.random.poisson(averagesize, tasksize // averagesize)
offsets = numpy.empty(len(counts) + 1, dtype=numpy.int32)
offsets[0] = 0
numpy.cumsum(counts, out=offsets[1:])
starts, stops = offsets[:-1], offsets[1:]

# content
content = numpy.random.normal(1, 0.00001, offsets[-1]).astype(numpy.float32)

# parents
parents = numpy.zeros(len(content), dtype=numpy.int32)
numpy.add.at(parents, offsets[offsets != offsets[-1]][1:], 1)
numpy.cumsum(parents, out=parents)

# mutable contents on GPU
scan = pycuda.gpuarray.to_gpu(content)

# flat array output
output = numpy.empty(len(starts), dtype=numpy.float32)

# run!
starttime = time.time()

d = 1
c = 0
while d < len(parents):
    reduce_a(numpy.int32(len(parents)), numpy.int32(d), In(parents), scan,
             block=(1024, 1, 1), grid=(len(content) // 1024 + 1, 1))
    d *= 2
    c += 1

reduce_b(numpy.int32(len(starts)), In(starts), In(stops), scan, Out(output),
         block=(1024, 1, 1), grid=(len(starts) // 1024 + 1, 1))

stoptime = time.time()

stoptime - starttime

1.4701042175292969